# Extract Data From PDF And Export CSV
Take at least 4 months worth of transactions and categorize them in excel after you extract the data and export the csv
You can use this as the training data for the predictions. 

In [114]:
import pdfplumber
import pandas as pd
import re

pd.set_option('display.max_colwidth', None)

# Read the PDF file  ---------Put file path here -----
with pdfplumber.open("Downloads\Statement_May_2023.pdf") as pdf:
    transactions = pd.DataFrame(columns=["Date", "Description", "Amount", "Extra Info"])
    
    for page in pdf.pages:
        texts = page.extract_text()
        
        def extract_transactions(texts):
            transaction_pattern = r'(\d{1,2}/\d{2})\s+(.*)\s(-?\d+\.\d{2}\-?)\n(\D*)'
            transactions = re.findall(transaction_pattern, texts)
            return transactions
   
        extraction = extract_transactions(texts)

        # Create a dataframe from the extracted data
        df = pd.DataFrame(extraction, columns=["Date", "Description", "Amount", "Extra Info"])
        df["Extraction"] = df['Description'].astype(str) +"-"+ df["Extra Info"]
        
        # put the extraction information in the description column
        df["Description"] = df["Extraction"]
        df = df.drop("Extra Info", axis=1)
        df = df.drop("Extraction", axis=1)
        

        # Sort the dataframe by date
        df = df.sort_values("Date")
        
        transactions = pd.concat([transactions, df], ignore_index=True)

# Save the merged dataframe to a CSV file
#                  ---put your file path here--- with what you want to call your file
transactions.to_csv("Desktop\May-2023-merged.csv", index=False)

# Print the merged dataframe
print(transactions)

     Date                    Description  Amount Extra Info
0    4/10   CASHOUT VENMO-MARIAH BAKER\n  235.00        NaN
1    4/10   CASHOUT VENMO-MARIAH BAKER\n  300.00        NaN
2    4/10              Deposit - Mobile-   60.00        NaN
3    4/10              Deposit - Mobile-  350.00        NaN
4    4/11       DEPOSIT INTUIT 69041465-  100.00        NaN
..    ...                            ...     ...        ...
209  5/09  TRANSFER BETTERMENT SEC-WEB A  50.00-        NaN
210  4/11     1,698.34 4/21 959.03 5/03-   32.73        NaN
211  4/12   1,852.08 4/24 1,801.97 5/04-   99.58        NaN
212  4/13   1,438.08 4/25 1,596.13 5/05-   93.58        NaN
213  4/20                   936.18 5/02-  136.73        NaN

[214 rows x 4 columns]


# Take Your CSV, Training Data, And Predict Your Categories 
Find your training data and put the file path in the df = pd.read_csv 
put another extracted file into the df2 = pd.read_csv that hasnt been categorized 

In [126]:
# Import the necessary libraries
import pdfplumber
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import ComplementNB
import csv



# df = the trainable data -- put your file path here 
df = pd.read_csv("Downloads\Transaction Training Data - Sheet2.csv")
# df2 = the test data -- put your file path here
df2 = pd.read_csv("Downloads\Feb - 2023 - Sheet2.csv")

# Create a dataframe from the extracted data
df = pd.DataFrame(df, columns=["Date", "Description", "Amount", "Extra Info", "Key Words"])
df2 = pd.DataFrame(df2, columns=["Date", "Description", "Amount", "Extra Info", "Key Words"])

# fill all empty data entries with "Empty"
df = df.fillna("Empty")
# df2 = df2.fillna("Empty")

# Extract the keywords from the Extra Info column
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(df["Description"])


# Create a training dataset
labels = df["Key Words"]

# Train a machine learning model
model = ComplementNB()
model.fit(features, labels)

# Use the model to predict the category of new transaction
predictions = []
for i in df2["Description"]:
    new_transaction = i
    new_features = vectorizer.transform([new_transaction])
    prediction = model.predict(new_features)
    predictions.append(prediction)
    
    
# Add predicitons to the dataframe
df2['Predicted Category'] = predictions
   

# # Save the merged dataframe to a CSV file
# #                  ---put your file path here--- with what you want to call your file
# df2.to_csv("Desktop\Predicted Categories.csv", index=False)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
# Print the prediction
    print(df2[["Predicted Category", "Key Words"]])

    Predicted Category    Key Words
0        [Credit Card]  Credit Card
1         [Eating out]   Eating out
2         [Eating out]   Eating out
3         [Eating out]   Eating out
4         [Eating out]   Eating out
5         [Eating out]   Eating out
6         [Eating out]   Eating out
7         [Eating out]   Eating out
8         [Eating out]   Eating out
9            [Monthly]      Monthly
10           [Monthly]      Monthly
11              [Food]         Food
12              [Food]         Food
13              [Home]         Home
14              [Home]         Home
15              [Food]         Food
16              [Food]         Food
17              [Food]         Food
18              [Food]         Food
19              [Misc]         Misc
20              [Misc]         Misc
21        [Eating out]   Eating out
22        [Eating out]   Eating out
23           [Monthly]      Monthly
24           [Monthly]      Monthly
25            [Income]       Income
26            [Income]      